# Coverage

On cherche à connaitre le % de couverture des oems en SoH

In [ ]:
from core.s3.s3_utils import *
import pandas as pd
from core.sql_utils import *


In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join oem o
            on o.id = vm.oem_id
           ;"""), con)

dbeaver_df.columns

In [ ]:
s3 = S3Service()
oems = ["bmw", "ford", "renault", "stellantis", "tesla", "tesla-fleet-telemetry", "volvo-cars", "volkswagen"] # bug "kia" \ "mercedez-benz" -> return 0 file

In [ ]:
keys = pd.DataFrame()
for oem in oems:
    print(oem)
    keys = pd.concat((keys, s3.list_responses_keys_of_brand(oem).assign(date=lambda df: df["file"].str[:-5].astype("datetime64[ns]"))))
keys['brand'] = keys['brand'].replace("BMW", "bmw").replace("tesla-fleet-telemetry", 'tesla')

`% de couverture totale`

In [ ]:
vin_with_soh = dbeaver_df.dropna(subset='soh')['vin'].unique()
keys[keys['vin'].isin(vin_with_soh)].shape[0] / keys.shape[0]  

### % de couverture par oem

In [ ]:
couverture = {}
for oem in ["bmw", "kia", "stellantis", "mercedes-benz", "volvo-cars", "volkswagen", "toyota", "ford", "renault", "mercedes", "volvo", "tesla"]:
    couverture[oem] = {}
    nbr_vin_with_data = keys[keys['brand']==oem].vin.nunique()
    vin_dbeaver_with_soh = dbeaver_df[dbeaver_df['oem_name']==oem].dropna(subset='soh')['vin'].unique()
    nbr_vin_with_soh = keys[keys['vin'].isin(vin_dbeaver_with_soh)].vin.nunique()
    
    couverture[oem]['nbr_vin'] = nbr_vin_with_data
    couverture[oem]['nbr_vin_with_soh'] = nbr_vin_with_soh
    couverture[oem]['coverage_oem'] = round(nbr_vin_with_soh * 100 /nbr_vin_with_data) if nbr_vin_with_data > 0 else nbr_vin_with_data
    print(f"pour {oem} on collecte de la donnée sur {nbr_vin_with_data} vin")
    print(f"pour {oem} on à au moins 1 SoH sur {nbr_vin_with_soh} vin")
    if nbr_vin_with_data > 0:
        print(f"Ca représente un couverture de {round(nbr_vin_with_soh *100 /nbr_vin_with_data)}% pour {oem}")
    else:
        pass
    print("\n")

In [ ]:
pd.DataFrame(couverture).T